## Comparison of execution of VerticalPvConfiguration and DefaultConfiguration

In this notebook, the values from the literature for phi, alpha, theta, and p_max are used.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd

from agrivoltaics_supply_side_management.agriculture.crops import Cultivation
from agrivoltaics_supply_side_management.configuration \
    import BifacialVerticalPvConfiguration, DefaultBifacialConfiguration
from agrivoltaics_supply_side_management.optimization.convex_optimization\
    import ConvexOptimization
from agrivoltaics_supply_side_management.photovoltaics.pv_modules\
    import BifacialElectricityGeneration
from agrivoltaics_supply_side_management.solar_irradiation.irradiance\
    import BifacialIrradianceManager

In [3]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
monocropping_phi, monocropping_alpha, monocropping_theta, monocropping_p_max = \
    0.03442841710379512, 0.09510809834054065, 1.327579142408395, 25.0

In [5]:
monocropping_net_photosynthetic_rate_parameters = {
                "phi": monocropping_phi,
                "alpha": monocropping_alpha,
                "theta": monocropping_theta,
                "p_max": monocropping_p_max
            }

In [6]:
intercropping_phi, intercropping_alpha, intercropping_theta, intercropping_p_max = \
    0.05498625745316387, 0.12013631354448669, 0.7442598071555129, 17.0

In [7]:
intercropping_net_photosynthetic_rate_parameters = {
                "phi": intercropping_phi,
                "alpha": intercropping_alpha,
                "theta": intercropping_theta,
                "p_max": intercropping_p_max
            }

In [8]:
posterior_intercropping_phi, posterior_intercropping_alpha, posterior_intercropping_theta, posterior_intercropping_p_max = \
    0.18, 0.14, -0.34, 17.0

In [9]:
posterior_intercropping_net_photosynthetic_rate_parameters = {
                "phi": posterior_intercropping_phi,
                "alpha": posterior_intercropping_alpha,
                "theta": posterior_intercropping_theta,
                "p_max": posterior_intercropping_p_max
            }

In [10]:
lattitude, longitude = 49.26757152616243, -123.25266177347093
timezone = 'Canada/Pacific'
time_range = pd.date_range('2022-07-06', '2022-07-07', freq='1T',
                             tz=timezone)

In [11]:
surface_azimuth = 180
surface_tilt = 20
axis_azimuth = 180
pvrow_height = 1
pvrow_width = 4
pitch = 10
gcr = pvrow_width / pitch
albedo = 0.2
n_pvrows = 3
index_observed_pvrow = 1
bifaciality = 0.75

In [12]:
bifacial_irradiance_manager = BifacialIrradianceManager(
    lattitude, longitude, timezone, time_range,
    surface_azimuth, surface_tilt, axis_azimuth, gcr,
    pvrow_height, pvrow_width, albedo, n_pvrows, index_observed_pvrow,
    bifaciality)

In [13]:
optimization = ConvexOptimization()

In [14]:
temp_model_parameters_type, module_name, inverter_name = (
    'open_rack_glass_glass', 'Trina_Solar_TSM_300DEG5C_07_II_',
    'ABB__MICRO_0_25_I_OUTD_US_208__208V_')

In [15]:
bifacial_electricity_generation = BifacialElectricityGeneration(
    lattitude, longitude, timezone,
    bifacial_irradiance_manager.bifacial_irradiances,
    temp_model_parameters_type,
    module_name, inverter_name,
    surface_tilt, surface_azimuth, 
    bifaciality)

In [16]:
harvest_index = 0.95
biomass_energy_ratio = 30
leaf_area_index = 5

In [17]:
default_crop_growth_regulating_factor = 0.95
default_cultivation = Cultivation(harvest_index, biomass_energy_ratio,
                    leaf_area_index, default_crop_growth_regulating_factor)

##### Take into account the difference in net photosynthetic rate at light saturation point

In [18]:
monocropping_net_photosynthetic_rate_at_light_saturation_point = 12.5
intercropping_net_photosynthetic_rate_at_light_saturation_point = 11.0

intercropping_crop_growth_regulating_factor =\
    default_crop_growth_regulating_factor * (
        intercropping_net_photosynthetic_rate_at_light_saturation_point /\
        monocropping_net_photosynthetic_rate_at_light_saturation_point)

In [19]:
intercropping_crop_growth_regulating_factor

0.836

In [20]:
intercropping_cultivation = Cultivation(harvest_index, biomass_energy_ratio,
                leaf_area_index, intercropping_crop_growth_regulating_factor)

In [21]:
ppfd_data = np.linspace(0, 1500)

### Default configuration

In [22]:
default_bifacial_configuration = DefaultBifacialConfiguration(
    monocropping_net_photosynthetic_rate_parameters, ppfd_data,
    bifacial_irradiance_manager, optimization, 
    bifacial_electricity_generation,
    default_cultivation)

In [23]:
default_electricity_supply, default_crop_yield, \
    default_cumulative_electric_power_for_morning_peak, \
    default_cumulative_electric_power_for_afternoon_peak \
        = default_bifacial_configuration.supply(time_range)

In [24]:
default_electricity_supply

1846.9871799090927

In [25]:
default_crop_yield

0.0027915385983769895

In [26]:
default_cumulative_electric_power_for_morning_peak

284.4088536020805

In [27]:
default_cumulative_electric_power_for_afternoon_peak

408.6670461434379

### Vertical configuration with prior values

In [28]:
bifacial_vertical_pv_configuration = BifacialVerticalPvConfiguration(
    intercropping_net_photosynthetic_rate_parameters, ppfd_data,
    bifacial_irradiance_manager, optimization, 
    bifacial_electricity_generation,
    intercropping_cultivation)

In [29]:
vertical_electricity_supply, vertical_crop_yield,\
    vertical_cumulative_electric_power_for_morning_peak, \
    vertical_cumulative_electric_power_for_afternoon_peak \
        = bifacial_vertical_pv_configuration.supply(time_range)

In [30]:
vertical_electricity_supply

1690.1325454027615

In [31]:
vertical_crop_yield

0.0028000904376386413

In [32]:
vertical_cumulative_electric_power_for_morning_peak

287.4827210603035

In [33]:
vertical_cumulative_electric_power_for_afternoon_peak

548.2925089959637

### Vertical configuration with posterior values

In [34]:
posterior_bifacial_vertical_pv_configuration =\
    BifacialVerticalPvConfiguration(
        posterior_intercropping_net_photosynthetic_rate_parameters, ppfd_data,
        bifacial_irradiance_manager, optimization, 
        bifacial_electricity_generation,
        intercropping_cultivation)

In [35]:
posterior_vertical_electricity_supply, posterior_vertical_crop_yield,\
    posterior_vertical_cumulative_electric_power_for_morning_peak, \
    posterior_vertical_cumulative_electric_power_for_afternoon_peak \
        = posterior_bifacial_vertical_pv_configuration.supply(time_range)

In [36]:
posterior_vertical_electricity_supply

1881.0909798283963

In [37]:
posterior_vertical_crop_yield

0.0022176375616646832

In [38]:
posterior_vertical_cumulative_electric_power_for_morning_peak

287.4827210603035

In [39]:
posterior_vertical_cumulative_electric_power_for_afternoon_peak

548.2925089959637

### Comparison

In [40]:
vertical_electricity_supply / default_electricity_supply

0.9150754070128134

In [41]:
vertical_crop_yield / default_crop_yield

1.0030634859452146

In [42]:
vertical_cumulative_electric_power_for_morning_peak / default_cumulative_electric_power_for_morning_peak

1.0108079176132951

In [43]:
vertical_cumulative_electric_power_for_afternoon_peak / default_cumulative_electric_power_for_afternoon_peak

1.3416606848292794

In [44]:
posterior_vertical_electricity_supply / default_electricity_supply

1.018464556922903

In [45]:
posterior_vertical_crop_yield / default_crop_yield

0.7944140779403966

In [46]:
posterior_vertical_cumulative_electric_power_for_morning_peak / default_cumulative_electric_power_for_morning_peak

1.0108079176132951

In [47]:
posterior_vertical_cumulative_electric_power_for_afternoon_peak / default_cumulative_electric_power_for_afternoon_peak

1.3416606848292794